In [2]:
import h2o
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 hours 28 mins
H2O cluster version:,3.16.0.4
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_bpinjar_zwsmcx
H2O cluster total nodes:,1
H2O cluster free memory:,6.874 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [3]:
retail_csv = "sample_set.csv"  # modify this for your machine


In [26]:

data = h2o.import_file(retail_csv,na_strings=["?"])  # 163,987 rows x 15 columns

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [27]:
data.shape

(1000000, 35)

In [28]:
data.isna().sum()

1776231.0

data.nacnt()

In [29]:
#na_omit_columns = data.filter_na_cols(frac=0.35)
#len(na_omit_columns)

In [30]:
data.isna().sum()

1776231.0

In [31]:
pandas_df = data.as_data_frame()

# Now we have pandas dataframe from H20 dataframe, we will separate categorical and numerical and work on Imputation

# We have to make sure that we drop rows before splitting the data so that since we are seperating numeric and categorical variables

# Extract only numeric variables from dataframe, but verify once to confirm using pandas_df.info  


Convert to categorical variables which are numeric

In [ ]:
#col_names = ['bedrooms','bathrooms','floors','waterfront','view','condition','grade','yr_built','zipcode']
#for col in col_names:
    #houseData[col] = houseData[col].astype('category',copy=False)

In [108]:
num_cols = pandas_df._get_numeric_data().columns

In [109]:
num_cols

Index([u'Row_ID', u'Household_ID', u'Var1', u'Var2', u'Var3', u'Var4', u'Var5',
       u'Var6', u'Var7', u'Var8', u'NVVar1', u'NVVar2', u'NVVar3', u'NVVar4',
       u'Claim_Amount'],
      dtype='object')

In [110]:
len(num_cols)

15

In [111]:
# not to consider Target value
num_cols  = num_cols.drop('Claim_Amount') 

In [112]:
num_cols

Index([u'Row_ID', u'Household_ID', u'Var1', u'Var2', u'Var3', u'Var4', u'Var5',
       u'Var6', u'Var7', u'Var8', u'NVVar1', u'NVVar2', u'NVVar3', u'NVVar4'],
      dtype='object')

#  To convert numeric variables to categorical

In [113]:
col_names = ['Calendar_Year','Model_Year','OrdCat','Vehicle']
for col in col_names:
    pandas_df[col] = pandas_df[col].astype('category',copy=False)

In [123]:
num_cols = pandas_df._get_numeric_data().columns
# not to consider Target value
num_cols  = num_cols.drop('Claim_Amount') 

In [124]:
num_cols

Index([u'Row_ID', u'Household_ID', u'Var1', u'Var2', u'Var3', u'Var4', u'Var5',
       u'Var6', u'Var7', u'Var8', u'NVVar1', u'NVVar2', u'NVVar3', u'NVVar4'],
      dtype='object')

In [125]:
#df_num.Model_Year.unique()
#df_num.OrdCat.unique()
#df_num.Calendar_Year.unique()
#df_num.Vehicle.value_counts()
#df_num.Calendar_Year.value_counts()

In [128]:
set_cols = pandas_df.columns
cat_cols = list(set(set_cols) - set(num_cols))

In [129]:
len(cat_cols)

21

In [132]:
type(cat_cols)
type(num_cols)

pandas.core.indexes.base.Index

In [133]:
cat_cols.remove('Claim_Amount')

In [134]:
cat_cols

['Cat9',
 'Blind_Submodel',
 'Blind_Model',
 'Cat7',
 'Calendar_Year',
 'Cat12',
 'Cat10',
 'Cat1',
 'Cat2',
 'Cat3',
 'Cat11',
 'Cat5',
 'Cat6',
 'Vehicle',
 'Cat8',
 'Cat4',
 'OrdCat',
 'NVCat',
 'Blind_Make',
 'Model_Year']

In [119]:
len(pandas_df.columns)

35

In [137]:
print len(cat_cols)
print len(num_cols)
print "--"
print len(pandas_df.columns)

20
14
--
35


In [138]:
# we will work on numeric data now

In [139]:
df_num = pandas_df[num_cols]

In [141]:
df_num.dtypes

Row_ID            int64
Household_ID      int64
Var1            float64
Var2            float64
Var3            float64
Var4            float64
Var5            float64
Var6            float64
Var7            float64
Var8            float64
NVVar1          float64
NVVar2          float64
NVVar3          float64
NVVar4          float64
dtype: object

In [142]:
#missing data
#Column wise
import pandas as pd
total = df_num.isnull().sum().sort_values(ascending=False)
percent = (df_num.isnull().sum()/df_num.isnull().count()).sort_values(ascending=False)
#percent
missing_data = pd.concat([total, percent*100], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)


,Total,Percent
NVVar4,0,0.0
NVVar3,0,0.0
NVVar2,0,0.0
NVVar1,0,0.0
Var8,0,0.0
Var7,0,0.0
Var6,0,0.0
Var5,0,0.0
Var4,0,0.0
Var3,0,0.0


In [143]:
df_num.isnull().sum().sum()

0L

In [144]:
df_num.isnull().sum(axis=1)

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
999970    0
999971    0
999972    0
999973    0
999974    0
999975    0
999976    0
999977    0
999978    0
999979    0
999980    0
999981    0
999982    0
999983    0
999984    0
999985    0
999986    0
999987    0
999988    0
999989    0
999990    0
999991    0
999992    0
999993    0
999994    0
999995    0
999996    0
999997    0
999998    0
999999    0
Length: 1000000, dtype: int64

# NA imputation 

In [145]:
# drop rows with missing values
#dataset.dropna(inplace=True)

In [146]:
# fill missing values with mean column values
#dataset.fillna(dataset.mean(), inplace=True)


# To drop columns

In [147]:
#df_num.drop(['id'],inplace=True,axis=1)

# To add a new column

In [148]:
#normalized_df['price'] = targetVar['price']

# To write to a csv file

In [149]:
#normalized_df.to_csv("numeric.csv")

In [150]:
#final_df = normalized_df.append(df_cat)

# To normalize

In [151]:
normalized_df=(df_num-df_num.mean())/df_num.std()

In [152]:
normalized_df.head()

,Row_ID,Household_ID,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,NVVar1,NVVar2,NVVar3,NVVar4
0,1.195258,1.141776,3.582030,0.832959,1.103290,1.171901,2.745342,2.538117,1.624347,-0.123385,-0.239522,-0.273228,-0.278539,-0.260585
1,1.440354,1.289121,-0.776329,-1.490486,-0.987780,-1.539403,-0.895255,-1.467846,-1.222086,-0.935044,-0.239522,-0.273228,-0.278539,-0.260585
2,-0.221916,-0.098996,2.186698,0.922322,1.466954,1.259362,1.982378,0.779605,1.245637,-0.172305,6.381809,-0.273228,-0.278539,-0.260585
3,1.133512,1.106828,-1.490410,-1.812194,-1.624192,-1.539403,-1.766355,-1.867128,-1.136570,-1.307623,-0.239522,2.658824,-0.278539,-0.260585
4,-1.663695,-1.734666,0.028039,0.386143,0.353232,0.297286,0.528542,0.018102,-0.672345,0.367127,-0.239522,-0.273228,-0.278539,-0.260585


In [153]:
#normalized_df['price'] = targetVar['price']
#pandas_df.Claim_Amount
normalized_df['Claim_Amount'] = pandas_df['Claim_Amount']

In [154]:
normalized_df.head()

,Row_ID,Household_ID,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
0,1.195258,1.141776,3.582030,0.832959,1.103290,1.171901,2.745342,2.538117,1.624347,-0.123385,-0.239522,-0.273228,-0.278539,-0.260585,0.0
1,1.440354,1.289121,-0.776329,-1.490486,-0.987780,-1.539403,-0.895255,-1.467846,-1.222086,-0.935044,-0.239522,-0.273228,-0.278539,-0.260585,0.0
2,-0.221916,-0.098996,2.186698,0.922322,1.466954,1.259362,1.982378,0.779605,1.245637,-0.172305,6.381809,-0.273228,-0.278539,-0.260585,0.0
3,1.133512,1.106828,-1.490410,-1.812194,-1.624192,-1.539403,-1.766355,-1.867128,-1.136570,-1.307623,-0.239522,2.658824,-0.278539,-0.260585,0.0
4,-1.663695,-1.734666,0.028039,0.386143,0.353232,0.297286,0.528542,0.018102,-0.672345,0.367127,-0.239522,-0.273228,-0.278539,-0.260585,0.0


# To get categorical data

In [155]:
df_cat = pandas_df[cat_cols]

In [156]:
df_cat.columns

Index([u'Cat9', u'Blind_Submodel', u'Blind_Model', u'Cat7', u'Calendar_Year',
       u'Cat12', u'Cat10', u'Cat1', u'Cat2', u'Cat3', u'Cat11', u'Cat5',
       u'Cat6', u'Vehicle', u'Cat8', u'Cat4', u'OrdCat', u'NVCat',
       u'Blind_Make', u'Model_Year'],
      dtype='object')

In [157]:
df_cat.to_csv("categorical.csv")
df_num.to_csv("numeric.csv")